<a href="https://colab.research.google.com/github/Y6HYUK/Tracking-tennis-ball-and-players/blob/main/Tracking_tennis_player_and_ball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Library
!pip install ultralytics
!pip install opencv-python
!pip install roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.5/913.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Download Dataset (YOLOv8) using Link : https://universe.roboflow.com/viren-dhanwani/tennis-ball-detection/dataset/6/download/yolov8

# Import the Roboflow library which provides tools to interact with Roboflow's API
from roboflow import Roboflow

# Initialize a Roboflow object with your unique API key
# Replace "----" with your actual Roboflow API key
rf = Roboflow(api_key="----")

# Access the specific workspace within your Roboflow account
# Replace "viren-dhanwani" with your workspace name if different
project = rf.workspace("viren-dhanwani").project("tennis-ball-detection")

# Select the specific version of your project you want to use
# In this case, version 6 is selected
version = project.version(6)

# Download the dataset in YOLOv8 format
# This will download the dataset to the default directory (usually '/content/datasets/' in Google Colab)
# The dataset includes images, labels, and the necessary configuration files for training
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tennis-ball-detection-6 in yolov8:: 100%|██████████| 1168/1168 [00:00<00:00, 1515.47it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import os

# Define the absolute path to the data.yaml file, which contains dataset configurations
data_yaml_path = os.path.join(dataset.location, "data.yaml")
print("data.yaml path:", data_yaml_path)

# Debugging purpose: Display the contents of data.yaml to ensure correct paths and settings
!cat {data_yaml_path}

# Initiate YOLOv8 training with specified parameters
!yolo detect train data={data_yaml_path} model=yolov8n.pt epochs=30 imgsz=640 batch=8 project="." name="tennis-ball-run"

# After training completes, the best performing model weights are saved at:
# ./tennis-ball-run/weights/best.pt


data.yaml path: /content/tennis-ball-detection-6/data.yaml
names:
- tennis ball
nc: 1
roboflow:
  license: CC BY 4.0
  project: tennis-ball-detection
  url: https://universe.roboflow.com/viren-dhanwani/tennis-ball-detection/dataset/6
  version: 6
  workspace: viren-dhanwani
test: ../test/images
train: ../train/images
val: ../valid/images
100% 6.25M/6.25M [00:00<00:00, 106MB/s]
Ultralytics 8.3.67 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/tennis-ball-detection-6/data.yaml, epochs=30, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=., name=tennis-ball-run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout

In [5]:
import cv2
import torch
from ultralytics import YOLO
import pickle

class PlayerTracker:
    """
    PlayerTracker is dedicated to tracking players (persons) in the video.
    - Utilizes YOLO's built-in tracker (`model.track`) to detect and track only 'person' class objects.
    - Maintains tracking for the first two detected players by fixing their IDs.
    """
    def __init__(self, model_path='yolov8m.pt'):
        self.model = YOLO(model_path)
        self.chosen_ids = None  # 처음 2명 track_id를 저장

    def detect_frames(self, frames, read_from_stub=False, stub_path=None):
        """
        Processes multiple video frames to detect and track players.

        Parameters:
        - frames (list): List of video frames to process.
        - read_from_stub (bool): If True, loads detection results from a pickle file instead of processing frames.
        - stub_path (str): Path to the pickle file for loading/saving detection results.

        Returns:
        - list: A list of dictionaries containing track IDs and bounding boxes for each frame.
        """
        if read_from_stub and stub_path is not None:
            with open(stub_path, 'rb') as f:
                return pickle.load(f)

        results_list = []
        for frame in frames:
            player_dict = self.detect_frame(frame)  # Detect players in the current frame
            results_list.append(player_dict)  # Append the detection results to the list

        if stub_path is not None:
            with open(stub_path, 'wb') as f:
                pickle.dump(results_list, f)

        return results_list

    def detect_frame(self, frame):
        """
        Detects and tracks players in a single video frame.

        Parameters:
        - frame (numpy.ndarray): The video frame to process.

        Returns:
        - dict: A dictionary mapping track IDs to their corresponding bounding boxes.
        """
        # Use YOLO's built-in tracker to detect and track objects in the frame
        # `persist=True` ensures that track IDs are maintained across frames
        results = self.model.track(frame, persist=True)[0]
        current_dict = {}
        for box in results.boxes:
            if box.id is None:
                continue
            track_id = int(box.id.tolist()[0])  # Extract the track ID
            cls_id = int(box.cls.tolist()[0])    # Extract the class ID
            cls_name = results.names[cls_id]     # Get the class name using the class ID

            # Only consider detections of the 'person' class
            if cls_name == "person":
                x1, y1, x2, y2 = box.xyxy.tolist()[0]  # Extract bounding box coordinates
                current_dict[track_id] = [x1, y1, x2, y2]  # Store in the dictionary

        # Logic to fix the first two detected players' IDs
        if self.chosen_ids is None:
            p_ids = list(current_dict.keys())  # Get list of current track IDs
            if len(p_ids) >= 2:
                self.chosen_ids = set(p_ids[:2])  # Fix the first two track IDs
            else:
                self.chosen_ids = set(p_ids)  # If less than two, fix all available track IDs

        # After fixing, retain only the detections with the chosen track IDs
        filtered_dict = {}
        for pid, bbox in current_dict.items():
            if pid in self.chosen_ids:
                filtered_dict[pid] = bbox

        return filtered_dict

    def draw_bboxes(self, frames, player_detections):
        """
        Draws bounding boxes and track IDs for detected players on each frame.

        Parameters:
        - frames (list): List of video frames.
        - player_detections (list): List of dictionaries containing player detections for each frame.

        Returns:
        - list: List of frames with bounding boxes and labels drawn.
        """
        out_frames = []
        for frame, p_dict in zip(frames, player_detections):
            for track_id, bbox in p_dict.items():
                x1,y1,x2,y2 = bbox
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)),
                              (0,0,255), 2)
                cv2.putText(frame, f"Player {track_id}",
                            (int(x1), int(y1)-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            out_frames.append(frame)
        return out_frames


class BallTracker:
    """
    BallTracker is dedicated to tracking the tennis ball in the video.
    - Utilizes a pre-trained YOLO model (`best.pt`) to detect and track only 'tennis ball' class objects.
    - Currently tracks a single ball by storing the most confident detection in each frame.
    """
    def __init__(self, model_path='best.pt'):
        self.model = YOLO(model_path)

    def detect_frames(self, frames, read_from_stub=False, stub_path=None):
        """
        Processes multiple video frames to detect and track the tennis ball.

        Parameters:
        - frames (list): List of video frames to process.
        - read_from_stub (bool): If True, loads detection results from a pickle file instead of processing frames.
        - stub_path (str): Path to the pickle file for loading/saving detection results.

        Returns:
        - list: A list of dictionaries containing track IDs and bounding boxes for each frame.
        """
        if read_from_stub and stub_path is not None:
            with open(stub_path,'rb') as f:
                return pickle.load(f)

        ball_results = []
        for frame in frames:
            ball_dict = self.detect_frame(frame)
            ball_results.append(ball_dict)

        if stub_path is not None:
            with open(stub_path,'wb') as f:
                pickle.dump(ball_results, f)

        return ball_results

    def detect_frame(self, frame):
        """
        Detects the tennis ball in a single video frame.

        Parameters:
        - frame (numpy.ndarray): The video frame to process.

        Returns:
        - dict: A dictionary mapping track IDs to their corresponding bounding boxes.
                Currently, it only tracks one ball with a fixed track ID of 1.
        """
        results = self.model.predict(frame, conf=0.3)[0]
        ball_dict = {}
        best_conf = 0
        for box in results.boxes:
            cls_id = int(box.cls.tolist()[0])
            cls_name = results.names[cls_id]
            conf_val = float(box.conf.tolist()[0])
            if cls_name in ["tennis ball"]:
                if conf_val > best_conf:
                    x1,y1,x2,y2 = box.xyxy.tolist()[0]
                    ball_dict[1] = [x1,y1,x2,y2]
                    best_conf = conf_val

        return ball_dict

    def draw_bboxes(self, frames, ball_detections):
        """
        Draws bounding boxes and track IDs for the detected tennis ball on each frame.

        Parameters:
        - frames (list): List of video frames.
        - ball_detections (list): List of dictionaries containing ball detections for each frame.

        Returns:
        - list: List of frames with bounding boxes and labels drawn.
        """
        out_frames = []
        for frame, b_dict in zip(frames, ball_detections):
            for track_id, bbox in b_dict.items():
                x1,y1,x2,y2 = bbox
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)),
                              (0,255,0), 2)  # 초록색
                cv2.putText(frame, f"Ball {track_id}", (int(x1), int(y1)-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            out_frames.append(frame)
        return out_frames


In [6]:
def main():
    import cv2  # Import OpenCV library for video processing

    # 1) Open the video file (video that tracks both players and the ball simultaneously)
    video_path = '/content/drive/MyDrive/tennis_ex.mp4'  # Path to the input video file
    cap = cv2.VideoCapture(video_path)  # Initialize video capture object
    frames = []  # Initialize a list to store all video frames

    # Read all frames from the video and store them in the frames list
    while True:
        ret, frame = cap.read()  # Read a frame from the video
        if not ret:
            break  # Exit the loop if no more frames are available
        frames.append(frame)  # Append the current frame to the frames list
    cap.release()  # Release the video capture object

    # 2) Initialize PlayerTracker with the YOLOv8m model for player detection
    player_tracker = PlayerTracker(model_path='yolov8m.pt')  # Path to the YOLOv8m model

    # 3) Initialize BallTracker with the trained best.pt model for tennis ball detection
    ball_tracker = BallTracker(model_path='/content/tennis-ball-run/weights/best.pt')  # Path to the trained tennis ball model

    # 4) Detect players in all frames, maintaining only the first two detected players
    player_detections = player_tracker.detect_frames(
        frames,
        read_from_stub=False,  # Do not read from a precomputed stub
        stub_path=None  # No stub path provided
    )

    # 5) Detect tennis balls in all frames
    ball_detections = ball_tracker.detect_frames(
        frames,
        read_from_stub=False,  # Do not read from a precomputed stub
        stub_path=None  # No stub path provided
    )

    # 6) Draw bounding boxes around detected players on each frame
    output_frames = player_tracker.draw_bboxes(frames, player_detections)

    # 7) Draw bounding boxes around detected tennis balls on each frame
    output_frames = ball_tracker.draw_bboxes(output_frames, ball_detections)

    # 8) Save the annotated frames as an output video file
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec (mp4v)
    fps = 30  # Frames per second for the output video (adjust to match input video FPS)
    w, h = frames[0].shape[1], frames[0].shape[0]  # Get width and height from the first frame
    out = cv2.VideoWriter('output.mp4', fourcc, fps, (w, h))  # Initialize video writer object

    # Write each annotated frame to the output video
    for f in output_frames:
        out.write(f)
    out.release()  # Release the video writer object

    print("Saved output.mp4")  # Notify the user that the video has been saved

# Entry point of the script
if __name__ == "__main__":
    main()


100%|██████████| 49.7M/49.7M [00:00<00:00, 327MB/s]


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 4.0s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 6 persons, 1 clock, 62.7ms
Speed: 4.0ms preprocess, 62.7ms inference, 634.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 clock, 24.8ms
Speed: 2.3ms preprocess, 24.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 clock, 24.7ms
Speed: 2.6ms preprocess, 24.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 clock, 24.8ms
Speed: 3.0ms preprocess, 24.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 clock, 24.7ms
Speed: 3.2ms preprocess, 24.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384,